# Extracting json data from rapid api using python.

In [4]:
import requests
import json

url = "https://covid-193.p.rapidapi.com/statistics"

headers = {
	"X-RapidAPI-Key": "1fe4c38a3bmshffba4fecb2b6aa5p1ea6cajsn087be965bacc",
	"X-RapidAPI-Host": "covid-193.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

data=response.json()

statistics=data['response']

statistics_json = json.dumps(statistics)

#print(statistics_json)

# Transforming raw json data using pandas.

In [5]:
import pandas as pd
covid_data=pd.read_json(statistics_json)
# covid_data.head()

In [6]:
# Adding new columns to separate the information in the columns cases, deaths and tests.

covid_data["total_cases"]=covid_data.loc[:, "cases"].apply(lambda x: x.get("total"))
covid_data["new_cases"]=covid_data.loc[:,"cases"].apply(lambda x:x.get("new"))
covid_data["total_deaths"]=covid_data.loc[:, "deaths"].apply(lambda x: x.get("total"))
covid_data["new_deaths"]=covid_data.loc[:,"deaths"].apply(lambda x:x.get("new"))
covid_data["total_tests"]=covid_data.loc[:, "tests"].apply(lambda x: x.get("total"))

In [7]:
covid_data.head()

,continent,country,population,cases,deaths,tests,day,time,total_cases,new_cases,total_deaths,new_deaths,total_tests
0,Africa,Saint-Helena,6115.0,"{'new': None, 'active': 2164, 'critical': None...","{'new': None, '1M_pop': None, 'total': None}","{'1M_pop': None, 'total': None}",2024-05-16,2024-05-16T06:30:13+00:00,2166,None,NaN,None,NaN
1,South-America,Falkland-Islands,3539.0,"{'new': None, 'active': 0, 'critical': None, '...","{'new': None, '1M_pop': None, 'total': None}","{'1M_pop': '2439107', 'total': 8632}",2024-05-16,2024-05-16T06:30:13+00:00,1930,None,NaN,None,8632.0
2,North-America,Montserrat,4965.0,"{'new': None, 'active': 19, 'critical': None, ...","{'new': None, '1M_pop': '1611', 'total': 8}","{'1M_pop': '3577442', 'total': 17762}",2024-05-16,2024-05-16T06:30:13+00:00,1403,None,8.0,None,17762.0
3,None,Diamond-Princess,NaN,"{'new': None, 'active': 0, 'critical': None, '...","{'new': None, '1M_pop': None, 'total': 13}","{'1M_pop': None, 'total': None}",2024-05-16,2024-05-16T06:30:13+00:00,712,None,13.0,None,NaN
4,Europe,Vatican-City,799.0,"{'new': None, 'active': 0, 'critical': None, '...","{'new': None, '1M_pop': None, 'total': None}","{'1M_pop': None, 'total': None}",2024-05-16,2024-05-16T06:30:13+00:00,29,None,NaN,None,NaN


In [8]:
# Dropping the columns with extra information.

covid_data.drop(['cases','deaths','tests'],axis=1,inplace=True)

In [9]:
covid_data.info(memory_usage="deep")
covid_data['total_tests'].max()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   continent     236 non-null    object 
 1   country       238 non-null    object 
 2   population    229 non-null    float64
 3   day           238 non-null    object 
 4   time          238 non-null    object 
 5   total_cases   238 non-null    int64  
 6   new_cases     17 non-null     object 
 7   total_deaths  233 non-null    float64
 8   new_deaths    3 non-null      object 
 9   total_tests   213 non-null    float64
dtypes: float64(3), int64(1), object(6)
memory usage: 84.1 KB


1186851502.0

In [10]:
covid_data.isna().sum()

continent         2
country           0
population        9
day               0
time              0
total_cases       0
new_cases       221
total_deaths      5
new_deaths      235
total_tests      25
dtype: int64

In [11]:
# Filling the missing values in multiple columns

covid_data_updated=covid_data.fillna({'new_cases':0,'total_deaths':0,'new_deaths':0,'total_tests':0,'population':0})

In [12]:
covid_data_updated.head()

,continent,country,population,day,time,total_cases,new_cases,total_deaths,new_deaths,total_tests
0,Africa,Saint-Helena,6115.0,2024-05-16,2024-05-16T06:30:13+00:00,2166,0,0.0,0,0.0
1,South-America,Falkland-Islands,3539.0,2024-05-16,2024-05-16T06:30:13+00:00,1930,0,0.0,0,8632.0
2,North-America,Montserrat,4965.0,2024-05-16,2024-05-16T06:30:13+00:00,1403,0,8.0,0,17762.0
3,None,Diamond-Princess,0.0,2024-05-16,2024-05-16T06:30:13+00:00,712,0,13.0,0,0.0
4,Europe,Vatican-City,799.0,2024-05-16,2024-05-16T06:30:13+00:00,29,0,0.0,0,0.0


In [13]:
# Standardizing column formats

covid_data_updated=covid_data_updated.astype(
    {"population":"int64",
     "total_deaths":"int64",
     "total_tests":"int64",
     "day":"datetime64[ns]",
     "time":"datetime64[ns]",
     "new_cases":"int16",
     "new_deaths":"int16"})


In [14]:
covid_data_updated.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   continent     236 non-null    object        
 1   country       238 non-null    object        
 2   population    238 non-null    int64         
 3   day           238 non-null    datetime64[ns]
 4   time          238 non-null    datetime64[ns]
 5   total_cases   238 non-null    int64         
 6   new_cases     238 non-null    int16         
 7   total_deaths  238 non-null    int64         
 8   new_deaths    238 non-null    int16         
 9   total_tests   238 non-null    int64         
dtypes: datetime64[ns](2), int16(2), int64(4), object(2)
memory usage: 42.3 KB


# Loading clean data in a SQL database (PostgresSQL) using psycopg2.

In [59]:
import psycopg2
import configparser

In [60]:
config=configparser.ConfigParser()
config.read_file(open("cluster.config"))

In [61]:
USER=config.get("DB","DB_USER")
PASSWORD=config.get("DB","DB_PASSWORD")
PORT=config.get("DB","DB_PORT")

In [15]:
# conn=psycopg2.connect(host="127.0.0.1", dbname="postgres", port=PORT, user=USER, password=PASSWORD)
# curr=conn.cursor()
# conn.set_session(autocommit=True)

# curr.execute("CREATE DATABASE IF NOT EXISTS COVID_DATA")
# conn.close()

In [62]:
conn=psycopg2.connect(host="127.0.0.1", dbname="covid_data", port=PORT, user=USER, password=PASSWORD)
curr=conn.cursor()

In [63]:
covid_data_table_create=("""CREATE TABLE IF NOT EXISTS covidData(
id SERIAL PRIMARY KEY,
continent varchar,
country varchar,
population int,
day TIMESTAMP,
time TIMESTAMP,
total_cases int,
new_cases int,
total_deaths int,
new_deaths int,
total_tests int
)""")
curr.execute(covid_data_table_create)
conn.commit()

In [64]:
covid_data_table_insert=("""INSERT INTO covidData(
continent, 
country, 
population, 
day, 
time, 
total_cases, 
new_cases, 
total_deaths, 
new_deaths, 
total_tests
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [65]:
for i,row in covid_data_updated.iterrows():
    curr.execute(covid_data_table_insert,list(row))

In [66]:
conn.commit()

In [67]:
conn.close()